In [26]:
import pandas as pd
import os
import glob as gb
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## ORB Detector


In [28]:
import cv2
def ORB(input_image,original_signatures):
  #Works well with images of different dimensions
  def orb_sim(img1, img2):
    # SURF is no longer available in cv2 so using ORB
    orb = cv2.ORB_create()

    # detect keypoints and descriptors
    kp_a, desc_a = orb.detectAndCompute(img1, None)
    kp_b, desc_b = orb.detectAndCompute(img2, None)

    # define the bruteforce matcher object
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
      
    #perform matches. 
    matches = bf.match(desc_a, desc_b)
    #Look for similar regions with distance < 60. Goes from 0 to 100 so pick a number between.
    similar_regions = [i for i in matches if i.distance < 60]  
    if len(matches) == 0:
      return 1
    return len(similar_regions) / len(matches)

  # img1 = cv2.imread('/content/drive/My Drive/Car.jpg', 0)  # 714 x 901 pixels
  # img3 = cv2.imread('/content/drive/My Drive/Heli.jpg', 0)  # 203 x 256 pixels


  #orb_similarity = orb_sim(original_signature, input_image)  #1.0 means identical. Lower = not similar
  max_identical = 0
  orb_similarity = 0
  for i in range(len(original_signatures)):
    original_signature = cv2.imread(original_signatures[i])
    orb_similarity = orb_sim(original_signature, input_image)
    if max_identical < orb_similarity:
      max_identical = orb_similarity


  #print("Similarity using ORB is: ", max_identical)
  if max_identical > 0.7:
    #print("The Signature geniune")
    return 1
  else:
    #print("The Signature is forged")
    return 0

## SIFT

In [29]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [30]:
def SIFT_sim(input_image,original_signatures):
  def SIFT(image1,image2):
    # Convert the training image to RGB
    training_image = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    testing_image = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    # Convert the training image to gray scale
    training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)
    testing_gray = cv2.cvtColor(testing_image, cv2.COLOR_RGB2GRAY)
    # # Display traning image and testing image
    # fx, plots = plt.subplots(1, 2, figsize=(20,10))
    # plots[0].set_title("Training Image")
    # plots[0].imshow(training_image)
    # plots[1].set_title("Testing Image")
    # plots[1].imshow(testing_image)
    #now checking whether the image matches using SIFT algorithm
    surf = cv2.SIFT_create()

    train_keypoints, train_descriptor = surf.detectAndCompute(training_gray, None)
    test_keypoints, test_descriptor = surf.detectAndCompute(testing_gray, None)

    bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck = False)

    # Perform the matching between the SURF descriptors of the training image and the test image
    matches = bf.match(train_descriptor, test_descriptor)

    # The matches with shorter distance are the ones we want.
    matches = sorted(matches, key = lambda x : x.distance)
    similar_regions=[i for i in matches if i.distance<1000 ]

    if(len(matches)==0):
        return 0
    return (len(similar_regions)/len(matches))
  
  max_identical = 0
  sift_similarity = 0
  for i in range(len(original_signatures)):
    original_signature = cv2.imread(original_signatures[i])
    sift_similarity = SIFT(original_signature, input_image)
    if max_identical < sift_similarity:
      max_identical = sift_similarity


  #print("Similarity using SIFT is: ", max_identical)
  if max_identical > 0.2:
    #print("The Signature geniune")
    return 1
  else:
    #print("The Signature is forged")
    return 0

# CNN Evaluation

In [31]:
genuine_dir = ["/content/drive/My Drive/Signature_classify/train","/content/drive/My Drive/Signature_classify/test"]
User1_images = []
for imgs in genuine_dir:
  files = gb.glob(pathname= str(imgs+ '/User1/*.png')) 
  User1_images.extend(files)
User1_images.append("/content/drive/My Drive/Signatures_detect/forge/02100001.png")
User1_images.append("/content/drive/My Drive/Signatures_detect/real/00100001.png")
print(User1_images)
print(len(User1_images))

User2_images = []
for imgs in genuine_dir:
  files = gb.glob(pathname= str(imgs+ '/User2/*.png')) 
  User2_images.extend(files)
User2_images.append("/content/drive/My Drive/Signatures_detect/forge/02100002.png")
User2_images.append("/content/drive/My Drive/Signatures_detect/real/00200002.png")
print(User2_images)
print(len(User2_images))

User3_images = []
for imgs in genuine_dir:
  files = gb.glob(pathname= str(imgs+ '/User3/*.png')) 
  User3_images.extend(files)
User3_images.append("/content/drive/My Drive/Signatures_detect/forge/02100003.png")
User3_images.append("/content/drive/My Drive/Signatures_detect/real/00300003.png")
print(User3_images)
print(len(User3_images))

User4_images = []
for imgs in genuine_dir:
  files = gb.glob(pathname= str(imgs+ '/User4/*.png')) 
  User4_images.extend(files)
User4_images.append("/content/drive/My Drive/Signatures_detect/forge/02100004.png")
User4_images.append("/content/drive/My Drive/Signatures_detect/real/00400004.png")
print(User4_images)
print(len(User4_images))

User5_images = []
for imgs in genuine_dir:
  files = gb.glob(pathname= str(imgs+ '/User5/*.png')) 
  User5_images.extend(files)
User5_images.append("/content/drive/My Drive/Signatures_detect/forge/02100005.png")
User5_images.append("/content/drive/My Drive/Signatures_detect/real/00500005.png")
print(User5_images)
print(len(User5_images))


['/content/drive/My Drive/Signature_classify/train/User1/00102001.png', '/content/drive/My Drive/Signature_classify/train/User1/00100001.png', '/content/drive/My Drive/Signature_classify/train/User1/00104001.png', '/content/drive/My Drive/Signature_classify/train/User1/00103001.png', '/content/drive/My Drive/Signature_classify/train/User1/00101001.png', '/content/drive/My Drive/Signature_classify/test/User1/00101001.png', '/content/drive/My Drive/Signature_classify/test/User1/00102001.png', '/content/drive/My Drive/Signature_classify/test/User1/00104001.png', '/content/drive/My Drive/Signature_classify/test/User1/00105001.png', '/content/drive/My Drive/Signature_classify/test/User1/00103001.png', '/content/drive/My Drive/Signatures_detect/forge/02100001.png', '/content/drive/My Drive/Signatures_detect/real/00100001.png']
12
['/content/drive/My Drive/Signature_classify/train/User2/00202002.png', '/content/drive/My Drive/Signature_classify/train/User2/00200002.png', '/content/drive/My Dr

In [38]:
all_users = []
all_users.append(User1_images)
all_users.append(User2_images)
all_users.append(User3_images)
all_users.append(User4_images)
all_users.append(User5_images)

model_path = "/content/drive/My Drive/Colab Notebooks/identify_sign.h5"
loaded_model = keras.models.load_model(model_path)

import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

correct_detection = 0
for I in range(len(all_users)):
  desired_output = I
  for j in all_users[I]:
    image = cv2.imread(j)
    image_fromarray = Image.fromarray(image, 'RGB')
    resize_image = image_fromarray.resize((128, 128))
    expand_input = np.expand_dims(resize_image,axis=0)
    input_data = np.array(expand_input)
    input_data = input_data/255

    pred = loaded_model.predict(input_data)
    result = pred.argmax()
    input_user = result
    if input_user == desired_output:
      correct_detection += 1
  
print("Accuracy of CNN: ",(correct_detection/60))

Accuracy of CNN:  0.95


# Evaluation

In [33]:
genuine_dir = ["/content/drive/My Drive/Signature_classify/train","/content/drive/My Drive/Signature_classify/test"]
genuine = []
for main in genuine_dir:
  for folder in os.listdir(main):
    files = gb.glob(pathname= str(main+ '/'+ folder + '/*.png')) 
    genuine.extend(files)
testing_genuine = ["/content/drive/My Drive/Signatures_detect/real/00100001.png",
                   "/content/drive/My Drive/Signatures_detect/real/00200002.png",
                   "/content/drive/My Drive/Signatures_detect/real/00300003.png",
                   "/content/drive/My Drive/Signatures_detect/real/00400004.png",
                   "/content/drive/My Drive/Signatures_detect/real/00500005.png"]
genuine.extend(testing_genuine)
print(genuine)
print(len(genuine))

['/content/drive/My Drive/Signature_classify/train/User1/00102001.png', '/content/drive/My Drive/Signature_classify/train/User1/00100001.png', '/content/drive/My Drive/Signature_classify/train/User1/00104001.png', '/content/drive/My Drive/Signature_classify/train/User1/00103001.png', '/content/drive/My Drive/Signature_classify/train/User1/00101001.png', '/content/drive/My Drive/Signature_classify/train/User4/00403004.png', '/content/drive/My Drive/Signature_classify/train/User4/00400004.png', '/content/drive/My Drive/Signature_classify/train/User4/00401004.png', '/content/drive/My Drive/Signature_classify/train/User4/00402004.png', '/content/drive/My Drive/Signature_classify/train/User4/00404004.png', '/content/drive/My Drive/Signature_classify/train/User5/00501005.png', '/content/drive/My Drive/Signature_classify/train/User5/00502005.png', '/content/drive/My Drive/Signature_classify/train/User5/00500005.png', '/content/drive/My Drive/Signature_classify/train/User5/00503005.png', '/con

In [34]:
forge = ["/content/drive/My Drive/Signatures_detect/forge/02100001.png",
          "/content/drive/My Drive/Signatures_detect/forge/02100002.png",
          "/content/drive/My Drive/Signatures_detect/forge/02100003.png",
          "/content/drive/My Drive/Signatures_detect/forge/02100004.png",
          "/content/drive/My Drive/Signatures_detect/forge/02100005.png"]
print(forge)
print(len(forge))

['/content/drive/My Drive/Signatures_detect/forge/02100001.png', '/content/drive/My Drive/Signatures_detect/forge/02100002.png', '/content/drive/My Drive/Signatures_detect/forge/02100003.png', '/content/drive/My Drive/Signatures_detect/forge/02100004.png', '/content/drive/My Drive/Signatures_detect/forge/02100005.png']
5


In [35]:
desired_output = 1 #genuine 
model_path = "/content/drive/My Drive/Colab Notebooks/identify_sign.h5"
loaded_model = keras.models.load_model(model_path)

import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

ORB_correct_genuine_detect = 0
SIFT_correct_genuine_detect = 0

for I in genuine:

  image = cv2.imread(I)

  image_fromarray = Image.fromarray(image, 'RGB')
  resize_image = image_fromarray.resize((128, 128))
  expand_input = np.expand_dims(resize_image,axis=0)
  input_data = np.array(expand_input)
  input_data = input_data/255

  pred = loaded_model.predict(input_data)
  result = pred.argmax()
  input_user = result + 1
  #print('The Signature Belongs to: User' + str(input_user)) 

  #original_signature = cv2.imread("/content/drive/My Drive/Signature_classify/train/User"+ str(input_user)+"/00"+str(input_user)+"0100"+str(input_user)+".png")
  original_signatures = []
  for i in range(5):
    original_signatures.append("/content/drive/My Drive/Signature_classify/train/User"+ str(input_user)+"/00"+str(input_user)+"0"+str(i)+"00"+str(input_user)+".png")

  if ORB(image,original_signatures) == desired_output:
    ORB_correct_genuine_detect += 1


  if SIFT_sim(image,original_signatures) == desired_output:
    SIFT_correct_genuine_detect += 1


print("ORB: ",ORB_correct_genuine_detect)
print("SIFT: ",SIFT_correct_genuine_detect)

ORB:  53
SIFT:  54


In [36]:
desired_output = 0 #forge 
model_path = "/content/drive/My Drive/Colab Notebooks/identify_sign.h5"
loaded_model = keras.models.load_model(model_path)

import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image

ORB_correct_forge_detect = 0
SIFT_correct_forge_detect = 0

for I in forge:

  image = cv2.imread(I)

  image_fromarray = Image.fromarray(image, 'RGB')
  resize_image = image_fromarray.resize((128, 128))
  expand_input = np.expand_dims(resize_image,axis=0)
  input_data = np.array(expand_input)
  input_data = input_data/255

  pred = loaded_model.predict(input_data)
  result = pred.argmax()
  input_user = result + 1
  #print('The Signature Belongs to: User' + str(input_user)) 

  #original_signature = cv2.imread("/content/drive/My Drive/Signature_classify/train/User"+ str(input_user)+"/00"+str(input_user)+"0100"+str(input_user)+".png")
  original_signatures = []
  for i in range(5):
    original_signatures.append("/content/drive/My Drive/Signature_classify/train/User"+ str(input_user)+"/00"+str(input_user)+"0"+str(i)+"00"+str(input_user)+".png")

  if ORB(image,original_signatures) == desired_output:
    ORB_correct_forge_detect += 1

  if SIFT_sim(image,original_signatures) == desired_output:
    SIFT_correct_forge_detect += 1


print("ORB: ",ORB_correct_forge_detect)
print("SIFT: ",SIFT_correct_forge_detect)

ORB:  5
SIFT:  4


In [37]:
print("Accuracy for ORB: ", ((ORB_correct_forge_detect+ORB_correct_genuine_detect)/60)) #60 - total images
print("Accruacy of SIFT: ",((SIFT_correct_forge_detect+SIFT_correct_genuine_detect)/60))

Accuracy for ORB:  0.9666666666666667
Accruacy of SIFT:  0.9666666666666667
